# Single Subject Segmentation Q1K

In [ ]:
# #use these commented parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
# subject_id = "1093S2"
# task_id = "TO"
# session_id = "01"
# run_id = "1"

#use these parameters when executing this notebook from the automation notebook..
subject_id = ""
task_id = ""
session_id = ""
run_id = ""


In [ ]:
# import packages
import mne
import mne_bids
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pathlib import Path
import shutil
import pylossless as ll
from autoreject import AutoReject

import warnings
warnings.filterwarnings('ignore')


# Set the parameters and read the pylossless data.

In [ ]:
project_path = "/project/def-emayada/q1k/experimental/"
pylossless_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
segment_path = "derivatives/segment/"

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path + sync_loss_path
)
print(bids_path)

In [ ]:
# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
#show channel types...
channel_types = eeg_raw.get_channel_types()
print("Channel Types:", channel_types)
print("Channel Names:", eeg_raw.info['ch_names'])

In [ ]:
# Define a few channel groups of interest and plot the data
frontal = ["E19", "E11", "E4", "E12", "E5"]
occipital = ["E61", "E62", "E78", "E67", "E72", "E77"]
# din = ["DIN"]
# pupil = ["pupil_left"]
# x_pos = ["xpos_left"]
# y_pos = ["ypos_left"]

#scale_dict = dict(eeg=1e-4, eyegaze=30, pupil=30)
scale_dict = dict(eeg=1e-4)

# picks must be numeric (not string) when passed to `raw.plot(..., order=)`
picks_idx = mne.pick_channels(eeg_raw.ch_names, frontal + occipital, ordered=True)
#picks_idx = mne.pick_channels(eeg_raw.ch_names, frontal + occipital, ordered=True)
eeg_raw.plot(start=0,duration=30,order=picks_idx, scalings=scale_dict)

# Segment the data to 'sv06' and 'sv15'

In [ ]:
# Get the events form the annotations
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)

## Relabel condition vars for niceness
eeg_event_dict['to/stimulus/audio/standard/set4/item1/SO41'] = eeg_event_dict.pop('SO41')
eeg_event_dict['to/stimulus/audio/standard/set4/item2/SO42'] = eeg_event_dict.pop('SO42')
eeg_event_dict['to/stimulus/audio/standard/set4/item3/SO43'] = eeg_event_dict.pop('SO43')
eeg_event_dict['to/stimulus/audio/standard/set4/item4/SO44'] = eeg_event_dict.pop('SO44')

eeg_event_dict['to/stimulus/audio/standard/set5/item1/SO51'] = eeg_event_dict.pop('SO51')
eeg_event_dict['to/stimulus/audio/standard/set5/item2/SO52'] = eeg_event_dict.pop('SO52')
eeg_event_dict['to/stimulus/audio/standard/set5/item3/SO53'] = eeg_event_dict.pop('SO53')
eeg_event_dict['to/stimulus/audio/standard/set5/item4/SO54'] = eeg_event_dict.pop('SO54')
eeg_event_dict['to/stimulus/audio/standard/set5/item1/SO55'] = eeg_event_dict.pop('SO55')

eeg_event_dict['to/stimulus/audio/standard/set6/item1/SO61'] = eeg_event_dict.pop('SO61')
eeg_event_dict['to/stimulus/audio/standard/set6/item2/SO62'] = eeg_event_dict.pop('SO62')
eeg_event_dict['to/stimulus/audio/standard/set6/item3/SO63'] = eeg_event_dict.pop('SO63')
eeg_event_dict['to/stimulus/audio/standard/set6/item4/SO64'] = eeg_event_dict.pop('SO64')
eeg_event_dict['to/stimulus/audio/standard/set6/item5/SO65'] = eeg_event_dict.pop('SO65')
eeg_event_dict['to/stimulus/audio/standard/set6/item6/SO66'] = eeg_event_dict.pop('SO66')

eeg_event_dict['to/stimulus/audio/standard/set7/item1/SO71'] = eeg_event_dict.pop('SO71')
eeg_event_dict['to/stimulus/audio/standard/set7/item2/SO72'] = eeg_event_dict.pop('SO72')
eeg_event_dict['to/stimulus/audio/standard/set7/item3/SO73'] = eeg_event_dict.pop('SO73')
eeg_event_dict['to/stimulus/audio/standard/set7/item4/SO74'] = eeg_event_dict.pop('SO74')
eeg_event_dict['to/stimulus/audio/standard/set7/item5/SO75'] = eeg_event_dict.pop('SO75')
eeg_event_dict['to/stimulus/audio/standard/set7/item6/SO76'] = eeg_event_dict.pop('SO76')
eeg_event_dict['to/stimulus/audio/standard/set7/item7/SO77'] = eeg_event_dict.pop('SO77')

eeg_event_dict['to/stimulus/audio/standard/set8/item1/SO81'] = eeg_event_dict.pop('SO81')
eeg_event_dict['to/stimulus/audio/standard/set8/item2/SO82'] = eeg_event_dict.pop('SO82')
eeg_event_dict['to/stimulus/audio/standard/set8/item3/SO83'] = eeg_event_dict.pop('SO83')
eeg_event_dict['to/stimulus/audio/standard/set8/item4/SO84'] = eeg_event_dict.pop('SO84')
eeg_event_dict['to/stimulus/audio/standard/set8/item5/SO85'] = eeg_event_dict.pop('SO85')
eeg_event_dict['to/stimulus/audio/standard/set8/item6/SO86'] = eeg_event_dict.pop('SO86')
eeg_event_dict['to/stimulus/audio/standard/set8/item7/SO87'] = eeg_event_dict.pop('SO87')
eeg_event_dict['to/stimulus/audio/standard/set8/item8/SO88'] = eeg_event_dict.pop('SO88')

eeg_event_dict['to/stimulus/audio/standard/set9/item1/SO91'] = eeg_event_dict.pop('SO91')
eeg_event_dict['to/stimulus/audio/standard/set9/item2/SO92'] = eeg_event_dict.pop('SO92')
eeg_event_dict['to/stimulus/audio/standard/set9/item3/SO93'] = eeg_event_dict.pop('SO93')
eeg_event_dict['to/stimulus/audio/standard/set9/item4/SO94'] = eeg_event_dict.pop('SO94')
eeg_event_dict['to/stimulus/audio/standard/set9/item5/SO95'] = eeg_event_dict.pop('SO95')
eeg_event_dict['to/stimulus/audio/standard/set9/item6/SO96'] = eeg_event_dict.pop('SO96')
eeg_event_dict['to/stimulus/audio/standard/set9/item7/SO97'] = eeg_event_dict.pop('SO97')
eeg_event_dict['to/stimulus/audio/standard/set9/item8/SO98'] = eeg_event_dict.pop('SO98')
eeg_event_dict['to/stimulus/audio/standard/set9/item9/SO99'] = eeg_event_dict.pop('SO99')

eeg_event_dict['to/stimulus/audio/deviant/set4/Dev4'] = eeg_event_dict.pop('Dev4')
eeg_event_dict['to/stimulus/audio/deviant/set5/Dev5'] = eeg_event_dict.pop('Dev5')
eeg_event_dict['to/stimulus/audio/deviant/set6/Dev6'] = eeg_event_dict.pop('Dev6')
eeg_event_dict['to/stimulus/audio/deviant/set7/Dev7'] = eeg_event_dict.pop('Dev7')
eeg_event_dict['to/stimulus/audio/deviant/set8/Dev8'] = eeg_event_dict.pop('Dev8')
eeg_event_dict['to/stimulus/audio/deviant/set9/Dev9'] = eeg_event_dict.pop('Dev9')




In [ ]:
eeg_event_dict

In [ ]:
#create a new dictionary keeping only keys that start with 'd' or 'g' ('display' and 'gaze' onset events)
epoch_eeg_event_dict = {key: value for key, value in eeg_event_dict.items() if key.startswith(('to/'))}
#print the filtered dictionary
#print(epoch_eeg_event_dict)

#epoch to the filtered dictionary .. select specific conditions at a later stage..
epochs = mne.Epochs(eeg_raw, eeg_events, event_id=epoch_eeg_event_dict, tmin=-1, tmax=1.0, on_missing='warn', event_repeated='drop')
#epochs = epochs[['dtbc', 'dtoc', 'dtgc']]
display(epochs)

##apply autoreject to the epochs..
#ar = AutoReject()
#epochs.load_data()
#epochs_clean = ar.fit_transform(epochs)

#save the cleaned epochs...
epochs.save(project_path + pylossless_path + sync_loss_path + segment_path + 'epoch_fif_files/TO/' + bids_path.basename + '_epo.fif', overwrite=True)

In [ ]:
#required_conditions = {'dtbc', 'dtoc', 'dtgc'}
#available_conditions = set(epochs.event_id.keys())
#if required_conditions.issubset(available_conditions):
#    print("All required conditions are present.")
#else:
#    print("Missing conditions:", required_conditions - available_conditions)

In [ ]:
# peak... at the EEG channel types
channel_types = epochs.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", epochs.info['ch_names'])

In [ ]:
#evokeds = {'dev': epochs['deviant'].average(picks=['eeg']), 'stan1': epochs['stan*item1'].average(picks=['eeg']), 'stan4': epochs['stan*item4'].average(picks=['eeg'])}
#mne.write_evokeds(project_path + pylossless_path + sync_loss_path + segment_path + 'erp_fif_files/TO/' + bids_path.basename + '_erp.fif',list(evokeds.values()), overwrite=True)

# Plot ERP envelopes and topographies

In [ ]:
# conditions = ['search/a5','search/a9','search/at']
# #conditions = ['dtgc','dtbc','dtoc']
# roi = ['E83']
# eye = ['pupil_left']
# decim = 2
# freqs = np.arange(2, 50, 2)
# n_cycles = freqs / 2

# epochs = epochs[conditions]
# epochs.apply_baseline((-.2, 0))


# # Get the original epochs data
# data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)

# # Get the data for the 'xpos_left' channel
# x_channel_index = epochs.ch_names.index('xpos_left')
# x_data = epochs.get_data(picks=['xpos_left'])
# x_baseline_mean = np.mean(x_data[:, :, 800:1000], axis=2, keepdims=True)  # Shape: (n_epochs, 1)
# #x_data_adjusted = np.abs(data - baseline_means)
# x_data_adjusted = x_data - x_baseline_mean

# # Get the data for the 'xpos_left' channel
# y_channel_index = epochs.ch_names.index('ypos_left')
# y_data = epochs.get_data(picks=['ypos_left'])  # Shape: (n_epochs, 2, n_times)
# #y_data = epochs.get_data()[:, y_channel_index, :]
# print ('y_data shape: ', y_data.shape)
# y_baseline_mean = np.mean(y_data[:, :, 800:1000], axis=2, keepdims=True)  # Shape: (n_epochs, 1)
# #x_data_adjusted = np.abs(data - baseline_means)
# y_data_adjusted = y_data - y_baseline_mean
# print ('y_data_adjusted shape: ', y_data_adjusted.shape)

# #absolute distance from baseline....
# origin_distance = np.sqrt(x_data_adjusted**2 + y_data_adjusted**2)
# print ('origin distance shape: ', origin_distance.shape)
# #first derivative of distance from origin..
# velocity = np.diff(origin_distance, axis=-1)
# velocity = np.pad(velocity, ((0, 0), (0, 0), (0, 1)), mode='constant', constant_values=0)

# #cumulative sum of velocity...
# total_travel = np.cumsum(velocity, axis=1)




# if np.any(np.isnan(x_baseline_mean)):
#     print("The array contains NaN values.")
# else:
#     print("The array does not contain any NaN values.")


# #insert the new signals into the data array.. 
# ch_id = epochs.ch_names.index('distance')
# data[:, ch_id, :] = origin_distance.squeeze(1)
# ch_id = epochs.ch_names.index('x_head')
# data[:, ch_id, :] = velocity.squeeze(1)
# ch_id = epochs.ch_names.index('y_head')
# data[:, ch_id, :] = total_travel.squeeze(1)

# # Create a new Epochs object with modified data but original info
# new_epochs = mne.EpochsArray(data, info=epochs.info, events=epochs.events, event_id=epochs.event_id, tmin=epochs.tmin)

# #rename the channels...
# # Define the mapping for renaming channels.. and rename the channel
# rename_mapping = {'distance': 'origin_dist'}
# new_epochs.rename_channels(rename_mapping)
# rename_mapping = {'x_head': 'velocity'}
# new_epochs.rename_channels(rename_mapping)
# rename_mapping = {'y_head': 'total_travel'}
# new_epochs.rename_channels(rename_mapping)


# epochs = new_epochs


In [ ]:
event_names = epochs.event_id.keys()
selected_events_stan1 = [event for event in event_names if "stan" in event and "item1" in event]
selected_events_stan4 = [event for event in event_names if "stan" in event and "item4" in event]
selected_events_deviant = [event for event in event_names if "deviant" in event and "Dev" in event]

evokeds = {'stan1': epochs[selected_events_stan1].average(picks=['eeg']), 'stan4': epochs[selected_events_stan4].average(picks=['eeg']), 'dev': epochs[selected_events_deviant].average(picks=['eeg','misc'])}

In [ ]:
evokeds['stan1'].plot_joint(picks=['eeg'], title='standard item 1')

In [ ]:
evokeds['stan4'].plot_joint(picks=['eeg'],title='standard item 4')

In [ ]:
evokeds['dev'].plot_joint(picks=['eeg'],title='deviant')

# Plot the ERP overlay

In [ ]:
# Plot ERP overlay
mne.viz.plot_compare_evokeds(evokeds, picks=['E62'], combine='mean')

In [ ]:
#epochs_condition_1 = epochs['sv06']
#epochs_condition_2 = epochs['sv15']
ch_name = 'E70'

decim = 2
freqs = np.arange(3, 50, 2)  # define frequencies of interest
n_cycles = freqs / 2

#epochs_chan = epochs.pick_channels([ch_name])


In [ ]:
decim = 2
freqs = np.arange(2, 50, 2)  # define frequencies of interest
n_cycles = freqs / 2

pow_1, itc_1 = mne.time_frequency.tfr_morlet(
    epochs[selected_events_stan1],
    freqs,
    picks=ch_name,
    n_cycles=n_cycles,
    decim=decim,
    return_itc=True,
    average=True,
)

pow_2, itc_2 = mne.time_frequency.tfr_morlet(
    epochs[selected_events_deviant],
    freqs,
    picks=ch_name,
    n_cycles=n_cycles,
    decim=decim,
    return_itc=True,
    average=True,
)

itc_dat_1 = itc_1.data[0, :, :]  # only 1 channel as 3D matrix
pow_dat_1 = pow_1.data[0, :, :]  # only 1 channel as 3D matrix

itc_dat_2 = itc_2.data[0, :, :]  # only 1 channel as 3D matrix
pow_dat_2 = pow_2.data[0, :, :]  # only 1 channel as 3D matrix

In [ ]:
times = 1e3 * epochs[selected_events_stan1].times  # change unit to ms

fig1, (ax1t, ax1b) = plt.subplots(2, 1, figsize=(6, 4))
fig1.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax1t.imshow(
    pow_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax1b.imshow(
    itc_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax1t.set_ylabel("Frequency (Hz)")
ax1t.set_title(f"target gap Induced power ({ch_name})")
ax1b.set_title(f"target gap Inter Trial Coherence ({ch_name})")
ax1b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
fig2, (ax2t, ax2b) = plt.subplots(2, 1, figsize=(6, 4))
fig2.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax2t.imshow(
    pow_dat_2,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax2b.imshow(
    itc_dat_2,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax2t.set_ylabel("Frequency (Hz)")
ax2t.set_title(f"target overlap Induced power ({ch_name})")
ax2b.set_title(f"target overlap Inter Trial Coherence ({ch_name})")
ax2b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
fig3, (ax3t, ax3b) = plt.subplots(2, 1, figsize=(6, 4))
fig3.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax3t.imshow(
    pow_dat_2 - pow_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax3b.imshow(
    itc_dat_2 - itc_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax3t.set_ylabel("Frequency (Hz)")
ax3t.set_title(f"gap - overlap Induced power ({ch_name})")
ax3b.set_title(f"gap - overlap Inter Trial Coherence ({ch_name})")
ax3b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{"exclude"}' --no-input --to html session_seg_vp.ipynb